In [2]:
#importing necessary libraries
import cv2
import numpy as np
import os

In [3]:
#path for label, configuration file and weight file for yolov3
label_path = "coco.names"
config_path = "yolov3.cfg"
weigth_path = "yolov3.weights"

In [4]:
#to extract the labels from the label file
labels = open(label_path).read().strip().split("\n")

#for preprocessing part
yoloShape = (416,416)

#to config and feed the weight to the network
network = cv2.dnn.readNetFromDarknet(config_path, weigth_path)

#to get layers
ln = network.getUnconnectedOutLayersNames()

In [5]:
#to initailise the camera
cam = cv2.VideoCapture(0)

while True:
    flag, img = cam.read()
    #preprocessing the image obtained
    new_img = cv2.dnn.blobFromImage(img, 1/255.0, yoloShape, swapRB = True)
    #feeding the new image to the network as input
    network.setInput(new_img)
    layer_outs = network.forward(ln)
    
    #loop for detection of object
    for output in layer_outs:
        for detection in output:
            scores = detection[5:]
            classId = np.argmax(scores)
            confidence = scores[classId]
            H,W = img.shape[:2]
            if float(confidence) > 0.8:
                Object = labels[classId]
                if Object == "person":
                    x,y,w,h = (detection[:4] * np.array([W , H, W, H])).astype("int")
                    (X, Y) = int(x-(w/2)), int(y-(h/2))
                    cv2.rectangle(img, (X,Y),(X+w, Y+h),(0,255,0),2)
                    cv2.putText(img,Object,(X,Y-20), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0,0,255), 2)
    cv2.imshow("Person Detection using YOLOv3",img)
    key = cv2.waitKey(10)
    if key == 27:
        break
cam.release()
cv2.destroyAllWindows()